## Contents
- [Introduction](#introduction)
    - [DSDP problem](#dsdp-problem)
    - [Policies](#policies)
- [Formal Definition](#formal-definition)
    - [Structural of DSDP](#structure-of-dsdp-input)
    - [Solution of DSDP](#solution-of-dsdp-output)
- [Value and Optimality (Property of DSDP)](#value-and-optimality)
    - [Two Operators](#two-operators)
    - [Principle of Optimality](#the-bellman-equation-and-the-principle-of-optimality)
- [Solving DSDP](#solving-dsdps)
    - [Value Funciton Iteration](#value-function-iteration)
    - [Policy iteration](#policy-iteration)

## Introduction

In the previous lecture, we had a brief introduction to the interesting aspects of DSDP through the McCall search model.

In this lecture, we will introduce the general structure of DSDP and discuss two methods for solving DSDP:
- Value Function Iteration
- Policy Function Iteration


### DSDP Problem

A discrete DP is a maximization problem with an objective function of the form

<a id='equation-dp-objective'></a>
$$
\sum_{t = 0}^{\infty} \beta^t \mathbb{E} r(s_t, a_t) \tag{1} \\
\text{s.t.} \quad a_t \in A(s_t)
$$

where

- $ s_t $ is the state variable  
- $ a_t $ is the action  
- $ A(s_t) $ is the set of feasible actions
- $ \beta $ is a discount factor  
- $ r(s_t, a_t) $ is interpreted as a current reward when the state is $ s_t $ and the action chosen is $ a_t $  

In the optimization problem, the expectation $\mathbb{E}$ is built on transition probabilities $ Q(s_t, a_t, s_{t+1}) $.

Each pair $ (s_t, a_t) $ pins down transition probabilities $ Q(s_t, a_t, s_{t+1}) $ for the next period state $ s_{t+1} $. **Thus, actions influence not only current rewards but also the future time path of the state.**

Examples:

- consuming today vs saving and accumulating assets  
- accepting a job offer today vs seeking a better one in the future  
- exercising an option now vs waiting  

### Policies

**The most fruitful way to think about solutions to discrete DP problems is to compare *policies*.**

In general, **a policy is a randomized map from past actions and states to current action.**

In the setting formalized below, it suffices to consider so-called **stationary Markov policies**, which consider only the current state.

In particular, a **stationary Markov policy is a time-invariant map $ \sigma $ from states to actions**

- $ a_t = \sigma(s_t) $ indicates that $ a_t $ is the action to be taken in state $ s_t $  

It is known that, for any arbitrary policy, there exists a stationary Markov policy that dominates it at least weakly.

Hereafter, stationary Markov policies are referred to simply as policies.

**The aim is to find an optimal policy, in the sense of one that maximizes [(1)](#equation-dp-objective).**

## Formal Definition

### Structure of DSDP (Input):

- A finite set of **states**: 
  $$ S = \{0, \ldots, n-1\} $$

- A finite set of **feasible actions**:
  $$ A(s), s \in S$$

- A **reward function** 
  $$ 
  r\colon \mathit{SA} \to \mathbb{R} \\
  \text{where $SA$ is set of feasible state-action pairs:} \\
  \mathit{SA} := \{(s, a) \mid s \in S, \; a \in A(s)\}
  $$  

- A **transition probability function** 
  $$ Q\colon \mathit{SA} \to \Delta(S) \\
  \text{where $ \Delta(S) $ is the set of probability distributions over $ S $.} $$  

- A **discount factor** 
  $$ \beta \in [0, 1) $$ 


### Solution of DSDP (Output):

- **Policy function**:  
  $$ \sigma\colon S \to A \\
  \text{where $A$ is the union of all feasible action sets:} \\
  A := \bigcup_{s \in S} A(s) = \{0, \ldots, m-1\}
  $$

- Policy $\sigma$ is called **feasible policy**  
  - If it maps states into feasible action set:
   $ \sigma(s) \in A(s), \forall s \in S $  
  - Define **set of all feasible polices** as $\Sigma$.

- If a decision-maker uses  a policy $ \sigma \in \Sigma $, then
  - the current reward at time $ t $ is $ r(s_t, \sigma(s_t)) $  
  - the probability that $ s_{t+1} = s' $ is $ Q(s_t, \sigma(s_t), s') $  

- For each $ \sigma \in \Sigma $, define
  - $ r_{\sigma} $ by $ r_{\sigma}(s) := r(s, \sigma(s)) $ 
  - $ Q_{\sigma} $ by $ Q_{\sigma}(s, s') := Q(s, \sigma(s), s') $  
    where $Q_{\sigma}$ is the probability matrix that satisfies:
    - $\sum_{s' \in S} Q_{\sigma}(s,s') = 1 $
    - $Q_{\sigma}(s,s')>0, \forall s,s' \in S$

- If we think of $ r_\sigma $ as a column vector, then so is $ Q_\sigma^t r_\sigma $, and the $ s $-th row of it can be expressed as

  <a id='equation-ddp-expec'></a>
  $$
  (Q_\sigma^t r_\sigma)(s) = \mathbb E [ r(s_t, \sigma(s_t)) \mid s_0 = s ]
  \quad \text{when } \{s_t\} \sim Q_\sigma \tag{2}
  $$
  
  - $\{s_t\} \sim Q_\sigma $ means that the state is generated by stochastic matrix $ Q_\sigma $.

### Value and Optimality

- Let $ v_{\sigma}(s) $ denote the discounted sum of expected rewards from policy $ \sigma $ when the initial state is $s$.
    - To calculate this quantity we use [(1)](#equation-dp-objective) and [(2)](#equation-ddp-expec) to get
    $$
    v_{\sigma}(s) = \sum_{t=0}^{\infty} \beta^t (Q_{\sigma}^t r_{\sigma})(s)
    \qquad (s \in S)
    $$

    - $v_{\sigma}(s)$ is called the **policy value function for the policy $ \sigma $**.

- The **optimal value function (value function)** $ v^*\colon S \to \mathbb{R} $ is defined by

    $$
    v^*(s) = \max_{\sigma \in \Sigma} v_{\sigma}(s) \qquad (s \in S)
    $$

- A policy $ \sigma \in \Sigma $ is called **optimal** if  
    $$ v_{\sigma}(s) = v^*(s), \forall s \in S $$

- Given any $ w \colon S \to \mathbb R $, a policy $ \sigma \in \Sigma $ is called **$ w $-greedy** if
    $$
    \sigma(s) \in \operatorname*{arg\,max}_{a \in A(s)}
    \left\{
        r(s, a) +
        \beta \sum_{s' \in S} w(s') Q(s, a, s')
    \right\}
    \qquad (s \in S)
    $$
    - As discussed in detail below, optimal policies are precisely those that are $ v^* $-greedy.

### Example of McCall Search Model

To deepen our understanding, we use the McCall model from the previous lecture as an example to observe how its elements correspond to the standard structure of DSDP.

#### Inputs
- **Set of states $ S $:**  
    - Possible values of wage offers: $\mathbb{W}$

- **Set of feasible actions $ A(s), s \in S $:**
    - For all $w \in \mathbb{W}$, $A(w) = \{ \text{Accept}, \text{Refuse} \}$

- **Reward function $r: SA \to \mathbb{R}$:**
    - $r(w, \text{Accept}) = w$
    - $r(w, \text{Refuse}) = c$

- **Transition probability function $Q: SA \to \Delta(S)$:**
    - $Q(w, \text{Accept}, w') = Q(w, \text{Refuse}, w') = q(w')$
    - Note that the distribution of wages is purely random and not affected by states and actions.

#### Outputs
- **Policy function $\sigma: S \to A$:**
    - $\sigma(w) = \text{Accept}$ if $w > \bar{w}$
    - $\sigma(w) = \text{Refuse}$ if $w < \bar{w}$
    - where $\bar{w}$ is the reservation wage

- **Value function for policy $v_{\sigma}: S \to \mathbb{R}$:**
    - $v_{\text{Accept}} = \frac{w}{1 - \beta}$
    - $v_{\text{Refuse}} = c + \beta \sum_{t=1}^{\infty} q(w_t) r(w_t,\sigma(w_t))$

- **Optimal value function $v^*: S \to \mathbb{R}$:**
    - $v^* = \max \{ v_{\text{Accept}}, v_{\text{Refuse}} \}$


### Two Operators

- The **Bellman operator** $ T\colon \mathbb{R}^S \to \mathbb{R}^S $
  is defined by
  $$
  (T v)(s) = \max_{a \in A(s)}
  \left\{
      r(s, a) + \beta \sum_{s' \in S} v(s') Q(s, a, s')
  \right\}
  \qquad (s \in S)
  $$

- For any policy function $ \sigma \in \Sigma $, the **policy operator** $ T_{\sigma}\colon \mathbb{R}^S \to \mathbb{R}^S $ is defined by  
  $$
  (T_{\sigma} v)(s) = r(s, \sigma(s)) +
      \beta \sum_{s' \in S} v(s') Q(s, \sigma(s), s')
  \qquad (s \in S)
  $$

  - This can be written more succinctly in operator notation as
    $$
    T_{\sigma} v = r_{\sigma} + \beta Q_{\sigma} v
    $$

- **Properties** (we skip trivial proofs)
  - **monotone**  
    $ v \leq w $  implies $ Tv \leq Tw $ pointwise on $ S $, and
    similarly for $ T_\sigma $  

  - **contraction mapping**  
    $ \lVert Tv - Tw \rVert \leq \beta \lVert v - w \rVert $ and similarly for $ T_\sigma $, where $ \lVert \cdot\rVert $ is the max norm  

  - For any policy $ \sigma $, its value $ v_{\sigma} $ is the **unique fixed point** of $ T_{\sigma} $.

### The Bellman Equation and the Principle of Optimality

The main principle of the theory of dynamic programming is that

- **the optimal value function $ v^* $ is a unique solution to the Bellman equation**  
    $$
    v(s) = \max_{a \in A(s)}
        \left\{
            r(s, a) + \beta \sum_{s' \in S} v(s') Q(s, a, s')
        \right\}
    \qquad (s \in S)
    $$
    or in other words, $ v^* $ is the unique fixed point of $ T $.

- **$ \sigma^* $ is an optimal policy function if and only if it is $ v^* $-greedy**  
    By the definition of greedy policies given above, this means that

    $$
    \sigma^*(s) \in \operatorname*{arg\,max}_{a \in A(s)}
        \left\{
        r(s, a) + \beta \sum_{s' \in S} v^*(s') Q(s, a, s')
        \right\}
    \qquad (s \in S)
    $$

## Solving DSDPs

Now that the theory has been set out, let’s turn to solution methods.

The code for solving discrete DPs is available in [ddp.py](https://quanteconpy.readthedocs.io/en/latest/markov/ddp.html) from the [QuantEcon.py](http://quantecon.org/quantecon-py) code library. `DiscreteDP`

It implements the three most important solution methods for discrete dynamic programs, namely

- value function iteration  
- policy function iteration  
- modified policy function iteration (skip for now)  


Let’s briefly review these algorithms and their implementation.

### Value Function Iteration

This algorithm uses the fact that the Bellman operator $ T $ is a contraction mapping with fixed point $ v^* $.

Hence, iterative application of $ T $ to any initial function $ v^0 \colon S \to \mathbb R $ converges to $ v^* $.

The `DiscreteDP` value iteration method implements value function iteration as
follows

1. Choose any $ v^0 \in \mathbb{R}^n $, and specify $ \varepsilon > 0 $; set $ i = 0 $.  
1. Compute $ v^{i+1} = T v^i $.  
1. If $ \lVert v^{i+1} - v^i\rVert <  [(1 - \beta) / (2\beta)] \varepsilon $,
  then go to step 4; otherwise, set $ i = i + 1 $ and go to step 2.  
1. Compute a $ v^{i+1} $-greedy policy $ \sigma $, and return $ v^{i+1} $ and $ \sigma $.  


Given $ \varepsilon > 0 $, the value iteration algorithm

- terminates in a finite number of iterations  
- returns an $ \varepsilon/2 $-approximation of the optimal value function and an $ \varepsilon $-optimal policy function (unless `iter_max` is reached)  


(While not explicit, in the actual implementation each algorithm is
terminated if the number of iterations reaches `iter_max`)

### Policy Iteration

This routine, also known as **Howard’s policy improvement algorithm**, exploits more closely the particular structure of a discrete DP problem.

Each iteration consists of
1. A policy evaluation step that computes the value $ v_{\sigma} $ of a policy $ \sigma $ by solving the linear equation $ v = T_{\sigma} v $.  
1. A policy improvement step that computes a $ v_{\sigma} $-greedy policy.  

The `DiscreteDP` policy iteration method runs as follows

1. Choose any $ v^0 \in \mathbb{R}^n $ and compute a $ v^0 $-greedy policy $ \sigma^0 $; set $ i = 0 $.  
1. Compute the value $ v_{\sigma^i} $ by solving
  the equation $ v = T_{\sigma^i} v $.  
1. Compute a $ v_{\sigma^i} $-greedy policy
  $ \sigma^{i+1} $; let $ \sigma^{i+1} = \sigma^i $ if
  possible.  
1. If $ \sigma^{i+1} = \sigma^i $, then return $ v_{\sigma^i} $
  and $ \sigma^{i+1} $; otherwise, set $ i = i + 1 $ and go to
  step 2.  


The policy iteration algorithm terminates in a finite number of
iterations.

It returns an optimal value function and an optimal policy function (unless `iter_max` is reached).